В этом ноутбуке мы используем библиотеку Tesseract для распознавания текста в файле PDF и на фото. Эта задача называется OCR, optical character recognition - Оптическое распознавание символов.

In [1]:
# Сначала надо поставить программы, от которых зависит библиотека pytesseract
# на Python. Эти программы ставятся операционной системой не только для работы
# в Python, поэтому используются команды apt-get и apt.
!apt-get install poppler-utils
!apt install tesseract-ocr
!apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (1,048 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124574 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd

In [2]:
# Теперь доставим саму библиотеку и несколько библиотек, от которых она зависит,
# используя команду pip.
!pip install pytesseract python-docx PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 74.4 MB/s eta 0:00:00


Для начала мы подгрузили все необходимые библиотеки и программы, конечно, если вы будете пользоваться чем-то кроме коллаба, вам придется устанавливать программы вручную

In [3]:
# Импортируем все необходимые библиотеки.
import fitz  # Это PyMuPDF, который мы ставили выше.
from PIL import Image
import pytesseract
from docx import Document
import io

In [4]:
# Это программа для подгрузки моделей OCR для русского языка.
!apt-get install tesseract-ocr-rus


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-rus
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,271 kB of archives.
After this operation, 3,877 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-rus all 1:4.00~git30-7274cfa-1.1 [1,271 kB]
Fetched 1,271 kB in 0s (4,586 kB/s)
Selecting previously unselected package tesseract-ocr-rus.
(Reading database ... 124784 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-rus_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-rus (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-rus (1:4.00~git30-7274cfa-1.1) ...


In [5]:
# Позже эти функции сослужат нам добрую службу.
# Но пока их можно просто бездумно встаить в код.

def filterInvalidXMLChars(string):
    """Фильтрует недопустимые символы XML из строки."""

    def isValidXMLChar(char):
        """Проверяет, является ли символ допустимым в XML."""
        c = ord(char)
        return (
            0x20 <= c <= 0xD7FF or
            0xE000 <= c <= 0xFFFD or
            0x10000 <= c <= 0x10FFFF or
            c in (0x9, 0xA, 0xD)
        )

    return ''.join(filter(isValidXMLChar, string))

In [6]:
# Как пример возьмем вот этот документ. Скачаем его командой wget и сохраним
# под именем sample.pdf.
!wget 'https://docs.google.com/uc?export=download&id=12sRmlDOly8X6CUR_V4Xfjrh052VlM6Jo' -O 'sample.pdf'

--2025-01-26 18:06:18--  https://docs.google.com/uc?export=download&id=12sRmlDOly8X6CUR_V4Xfjrh052VlM6Jo
Resolving docs.google.com (docs.google.com)... 108.177.12.139, 108.177.12.113, 108.177.12.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=12sRmlDOly8X6CUR_V4Xfjrh052VlM6Jo&export=download [following]
--2025-01-26 18:06:18--  https://drive.usercontent.google.com/download?id=12sRmlDOly8X6CUR_V4Xfjrh052VlM6Jo&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.203.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181046 (177K) [application/octet-stream]
Saving to: ‘sample.pdf’

sample.pdf          100%[===================>] 176.80K  --.-KB/s    in 0

In [7]:
# Инициализация документа
doc = Document()

# Открываем PDF
pdf_document = "sample.pdf"
doc_pdf = fitz.open(pdf_document)

for page_number in range(doc_pdf.page_count):
    page = doc_pdf.load_page(page_number)
    pix = page.get_pixmap()

    # Конвертируем страницу в изображение
    img = Image.open(io.BytesIO(pix.tobytes("png")))


    # Укажите нужный язык "lang=''", например 'rus' для русского.
    # Нам нужно подгрузить модель для распознования. Для этого нужно узнать
    # код языка (https://tesseract-ocr.github.io/tessdoc/Data-Files-in-different-versions.html)
    # Выше мы уже скачали нужную модель при помощи команды
    # !apt-get install tesseract-ocr-(код языка, например tesseract-ocr-rus).

    # Теперь вызываем Тессеракт.
    # Через '+' можно добавить распознование конкретных языков, если знаете,
    # какие языки в тексте.
    text = pytesseract.image_to_string(img, lang='eng+rus')

    # Добавляем текст на новую страницу в файле DOCX.
    # add_paragraph может начать ругаться на получившийся результа и выдать ошибку:
    # "All strings must be XML compatible: Unicode or ASCII, no NULL bytes or control characters"
    # Так происходит из-за того, что OCR не смог корректно распознать символы.
    # Поэтому мы добавили ту самую функцию filterInvalidXMLChars, которую инициализировали выше.
    doc.add_paragraph(filterInvalidXMLChars(text))

# Сохраняем полученный документ. Его можно найти на панели слева - в файлах.
doc.save("output.docx")

In [8]:
# А теперь посмотрим, что за текст мы получили. Не идеально, но кто же идеален?
text

"O that you were yourself, ut love you are\nNo longer yours, than you your self here lve,\n‘gains this coming end you should prepare,\n‘and your sweet semblance to some ather give.\nSo should that beauty which you hold in lease\nFind no determination, then you were\n\nYour self again after your sels decease,\n\n\\When your sweet issue your sweet form should\nbear.\n\nWho lets so fair @ house fall to decay,\n\n\\Which husbandry in honour might uphold,\nAgainst the stormy gusts of winters day\n\n‘and barren rage of death's eternal cold?\n\nCO none but unthrits, dear my love you know,\nYou had a father, lt your son says.\n\nНе изменяйся, будь самим собой.\nТы можешь быть собой, пока жива\nKorga me смерть разрушит образтвой\nПусть будет итото на тебя похом.\n\n \n\nТебе природой красота дана\nНа очень кратний сроки потому\nПускай поправу перейдет она\nнаследнику прямому твоему,\n\nзаботливых руках прекрасный дом.\nHe aporuer перед натиском зимы,\n\nИ никогда не воцарится в нем\n`Дыханье см